# Observation Planning Part 2 - Sensitivity Calculation

1. Where is my target?
2. When can I observe my target?
3. How do I know when I've found my target (make a finder chart)?
4. __How long do I need to observe?__
5. How will I calibrate my data?

In Observation Planning part 1, we covered parts 1,2,3 above.  Now we'll cover the 4th part, calculating observation times.

First, a brief review:  For each of the following questions, give a brief explanation of what tools you use to answer the first three questions:

(1) Where is my target?

Your answer here

(2) When can I observe my target?

Your answer here

(3) How do I know when I've found my target?


Your answer here

## How long do I need to observe my target?

We know approximately how long we _can_ observe each target, but how long do we _need_ per source?  To answer this question, we need to do some sensitivity calculations.


First, we determine how bright our target is.  You can only obtain this information for known sources, but generally, we can set a reasonable limit based off of physical considerations.  In other words, as long as we know what we're looking for, we can make a pretty good guess of how long we need to look.

Our observations will be performed in the visual, meaning we are interested in the source brightness in the R (red) and V (visual) filters.

We can use SIMBAD to look up the typical brightness of the target source in magnitudes.  You can go on the [SIMBAD website](http://simbad.u-strasbg.fr/simbad/) to see how this works, but astroquery can also do it, so we're going to use astroquery here.  The astroquery SIMBAD module tells you what "fields" (data column names) are available to query on for each source.

In [3]:
from astroquery.simbad import Simbad

Recall from the previous workbook that we can use SIMBAD to retrieve information about stars, but the information included does not, by default, include the flux (or magnitude) of the source.

In [ ]:
pcyg_simbad = Simbad.query_object('P Cygni')
pcyg_simbad

In [ ]:
Simbad.list_votable_fields()

We add the flux fields for the R and V filters, then we can see the two new columns added on the right:

In [ ]:
Simbad.reset_votable_fields()
Simbad.add_votable_fields('flux(R)', 'flux(V)')
pcyg_simbad = Simbad.query_object('P Cygni')
pcyg_simbad

P Cygni is about 4.3 magnitudes in R.  What does that mean?

## Magnitudes


Magnitudes are a weird system with relative definitions.  The definition of magnitude is:

$$ m = -2.5 \log(F / F_0) $$

$m$ is the observed (apparent) magnitude, $F$ is the source flux (in units of Jansky, ergs/cm$^2$/angstrom, W/m$^2$/angstrom, or erg/cm$^{2}$/Hz, usually), and $F_0$ is the flux at magnitude zero. 

__$F_0$ is a definition!__

It was originally referenced to Vega, but more accurate observations have revealed that Vega is itself variable, so now the absolute standard is set by physical units.

So, we have to look up $F_0$ or determine it ourselves.

Note that each filter will have its own zero-point (also, terminology aside: the "zero point of a filter" and the "photometric zero point" are different things).  For example, for the visible $V$ band, we say:

$$ m_V = -2.5 \log(F_V / F_{0,V}) $$
or for red:
$$ m_R = -2.5 \log(F_R / F_{0,R}) $$

In the red filter, the zero-point is approximately 2500 Jy (which we just look up).  Therefore, we can compute the flux from P Cygni:

$$ m_{P Cyg} = 4.3 = -2.5 \log(F_{P Cyg} / 2500 \mathrm{Jy}) $$

Solve for $F_{P Cyg}$:

$$ F_{P Cyg} = 10^{(-4.3/2.5)} \times 2500 \mathrm{Jy} $$

In [4]:
from astropy import units as u
from astropy import constants
import numpy as np

In [ ]:
F_PCyg = 10**(-4.3/2.5) * 2500*u.Jy
F_PCyg

This tells us the amount of energy received per area per unit frequency at the telescope.  Next, we need to compute the number of photons we'll collect.

1 Jansky is $10^{-23}$ ergs s$^{-1}$ cm$^{-2}$ Hz$^{-1}$ (https://en.wikipedia.org/wiki/Jansky)

In [ ]:
(1*u.Jy).to(u.erg/u.s/u.cm**2/u.Hz)

## Modeling the telescope

We need to describe our instrument now.  Our telescope is a 14" diameter telescope, so its area is (pi * (radius)$^2$).  An inch is 2.54 centimeters.  So, our collecting area is $A_{CTO 14"}=$

In [ ]:
# student answer here
A_CTO = answer here
A_CTO

This should be 993.14666 cm$^2$

## Modeling the Filter

To determine how many photons we will receive, we need to know which wavelengths of photons are hitting the detector, and which ones are energetic enough to produce electrons through the photoelectric effect.

We do this by putting a filter in front of the detector, which prevents longer- or shorter-wavelength (lower- or higher-energy) photons from getting through.

We will approximate that our red filter is $\sim$1000 angstroms wide (in reality, there is a [wavelength-dependent filter function we could use](http://voservices.net/filter/), but we will take the simple approach for now).  The width of a filter is relative to the wavelength, and the red filter is centered at about 6500 angstroms.

To convert a central wavelength from angstroms to Hertz, we use the definition $\nu  \lambda = c$, i.e., the frequency times the wavelength is the speed of light. 

For example, 100 microns ($\mu$m) is at 3 THz.  Astropy's `constants.c` is the speed of light:

In [ ]:
wavelength = 100*u.um
frequency = constants.c  / wavelength
frequency

Note the units on that are something funky, $m \mu^{-1} s^{-1}$.  You can use astropy's `.to` to convert to a specific unit, or `.decompose` to reduce to a minimal expressible unit.  For example:

In [ ]:
print("Decomposed frequency: ", frequency.decompose())
print("Frequency in THz: ", frequency.to(u.THz))

So, returning to our red filter, for a wavelength $\lambda = 6500$ angstroms, solve the equation $\lambda \nu = c$ for $\nu$ in Hz or THz:

In [ ]:
# student answer here
nu = answer here
nu

Should be $4.6121917×10^{14}$ Hz

The filter width is $\Delta\lambda / \lambda$.  We have defined the width $\Delta \lambda = 1000$ angstroms, and the central wavelength is $\lambda=6500$ angstroms.  The fractional width of the filter is the same in both units, i.e.:
$$ \frac{\Delta\lambda}{\lambda} = \frac{\Delta\nu}{\nu} $$


You solved for $\nu$ above, so now determine $\Delta \nu$ (again, in Hz or THz)

In [ ]:
# student answer here
dlam = *u.AA
lam = *u.AA # note we can't use the variable "lambda" because it's a reserved word in python
dnu = answer here
dnu

Your answer should be $7.0956795×10^{13}$Hz

We can convert our flux in Janskys to $10^{-23}$ ergs s$^{-1}$ cm$^{-2}$ Hz$^{-1}$, then multiply by the bandwidth in Hz and the collecting area in cm$^{2}$ to obtain the energy we will receive on the detector per second:

$$ E_{rec} = F_{P Cyg} * A_{CTO 14"} * \Delta \nu $$

Compute $E_{rec}$ in units of erg $s^{-1}$ (`u.erg/u.s`):

In [ ]:
# you should have these three variables defined already
dnu, F_PCyg, A_CTO

In [ ]:
# student answer here (in erg/s)
E_rec = answer here
E_rec<you need to convert units>

Should be: $3.3569694×10^{−5}$ ergs / s

The CCD detector responds to individual photons, so we need to count how many photons are coming in.  The average wavelegnth of a photon is 6500 angstroms (approximately).  We can compute the energy per photon using

$$ E_{phot} = h \nu$$

where $h$ is [Planck's constant](https://en.wikipedia.org/wiki/Planck_constant).  Using the above information, compute the mean energy per photon.

(recall that frequency and wavelength are related as $c = \nu / \lambda$)

In [ ]:
# student answer here (compute the anwer in both eV and erg)
E_phot = <answer here>
E_phot<in eV>, E_phot<in erg>

You should obtain a number near 2 eV.

Given our energy received from the star $F_{E} =$ `E_rec` [erg/s] and the energy per photon $E_{phot}$ = `E_phot`[erg], how many photons are we receiving per second $F_{\gamma}$?

$F_{\gamma} [\mathrm{phot~s}^{-1}] = F_{E} / E_{phot}$

In [ ]:
# student answer here (units (1/u.s))
phot_per_s = answer here
phot_per_s

Answer should be: 10984594 s$^{-1}$

CCD detectors saturate at approximately $2^{16} = 65536$ counts (though ours saturate at closer to 30,000).  Is the photon count greater than or less than this number? 

student answer

If the above is greater than the saturation count, how many pixels do we need to spread the photons over if we want the image to stay below the saturation limit in a 1 second exposure?

In [ ]:
# student answer
npix = answer here
npix

The CCDs at CTO are ST-402 cameras.  They have 765x510 pixels over 6.9x4.3mm.

How big are the individual pixels, in microns? (should be about 9x9 microns)

In [ ]:
# student answer: compute the pixel size in both dimensions
answer here (there are two dimensions)

By observing a pair of stars with known separation on the sky, we can determine what the _pixel scale_ of our CCD is.  

However, for the purpose of this exercise, we will simply assume that the pixel scale is $\approx0.5"$ per pixel.

Over what radius circle does our star need to be projected to avoid saturating any pixels?

We calculate this in two steps:  First, we know that the _area_ we need to project onto must be ~168 pixels.  What _radius_ circle has an area of 168 pixels?

In [ ]:
# student answer
radius_pixels = answer here
radius_pixels

Should be 7.3 pixels

What does that size in pixels correspond to in arcseconds?

In [ ]:
# the "pixel scale" is the number of arcseconds across a pixel,
# so the units are "arcseconds per pixel",
# but we leave the "per pixel" part implicit (otherwise the code gets confusing)
pixel_scale = 0.5*u.arcsec

In [ ]:
radius_pixels * pixel_scale

Is the radius we need to smear the source over, i.e., the radius in arcseconds we just calculated above, bigger or smaller than the expected source size (the "point spread function") given the typical seeing conditions at CTO ($\sim2"$)?

your answer here

Given that answer, can we observe this target without saturating the CCD?  Justify your answer.

your answer here

## Integration Times

In the above, we were simply determining whether the source would be above or below the saturation level of our CCDs.  Let's now determine how long we need to integrate to achieve a target signal to noise ratio.

We decide our integration time based on a target signal-to-noise ratio (SNR).  If our target SNR is 100, how long do we need to integrate?

To a first approximation, we can assume that the signal-to-noise is limited by counting statistics.

The signal $S$ rises as the count rate $\dot{p}$ times the time,  $S = \dot{p} t$.

The noise $N$ is, in the Poisson noise limit, the square root of the signal (in photons), i.e., $N = \sqrt{S} = \sqrt{\dot{p}t}$.  The SNR is therefore 
$$S/N = \frac{\dot{p} t}{\sqrt{\dot{p} t}} = \sqrt{\dot{p} t}$$

## Warning / Note

When we say that the uncertainty is the square root of the signal when counting photons, we have a bit of a units problem!

$$ \sigma = \sqrt{\lambda}$$

The number of photons counted has units: number of photons.  The _uncertainty_ on that number has the same units, despite the fact that, naively, you would look at the above equation and say that the uncertainty has units of $\sqrt{counts}$.

You can't rely on the astropy unit system (or any automated unit system) to handle this equivalency; you need to take care to establish the correct units when dealing with uncertainties on counts.

We calculated the count rate above ($\dot{p}$ = `phot_per_s`), so what integration time $t$ do we need for $SNR=100$ ?

(solve the above equation, $SNR = \sqrt{\dot{p} t}$, for $t$)

In [ ]:
# student answer (in seconds)
integration_time = answer here (in seconds)
integration_time

You should note that that is a pretty short time! (about 0.9 milliseconds)

What if we account for read noise?  Let's assume that we are averaging over a 2 arcsecond Gaussian PSF.  First, how many pixels is that?

The area of a Gaussian isn't exactly that of a circle.  $A_{2D \mathrm{Gaussian}} = 2 \pi \sigma^2$

In [ ]:
psf_area = 2 * np.pi * (2*u.arcsec)**2
psf_area

In [ ]:
psf_area_pixels = psf_area * pixel_scale**-2
psf_area_pixels

Given that our readnoise is 10 counts per pixel, and we're interested in the _sum_ over these pixels, what is the expected contribution to the noise on the sum from these pixels?

Remember the propagation of error formula for summation:
$$sum = \Sigma x_i$$
$$\sigma_{sum}^2 = \Sigma_i \sigma_i^2$$
$$\sigma_{sum} = \sqrt{\Sigma_i \sigma_i^2}$$

In [ ]:
# student answer
readnoise_per_pix = 10*u.adu/u.pix
readnoise_sum = answer here
readnoise_sum

By coincidence, that's about 100.265 ADU

Given our calculation of the SNR and count noise above, what's our expected _total_ noise in the measurement over the PSF? (recall: the signal will be $t \times \dot{p}$, and we have calculated both of those quantities above)

In [ ]:
# student answer (note: this value should be in the same units as the read noise above - you might have to add those units!)
photon_noise = answer here
photon_noise

In [ ]:
# student answer
total_noise = answer here
total_noise

Should be about sqrt(2) * 100

Given that our noise is higher than we originally thought, we will not achieve our target signal to noise ratio if we use the `integration_time` above!  What will we achieve instead?

In [ ]:
# student answer
signal = answer here
snr = answer here
snr

We can write down a different equation for the signal-to-noise ratio now that we know how much readnoise to expect for a stellar observation.

We have just computed that our total noise is $N = {\sqrt{\dot{p} t + \sigma_{rn}^2}}$, so our SNR is:

$$S/N = \frac{\dot{p} t}{\sqrt{\dot{p} t + \sigma_{rn}^2}} \approx \frac{\dot{p} t}{\sqrt{\dot{p} t + 100}} $$

This equation is no longer trivially solvable, but it can be rearranged nicely enough into a quadratic equation:
$$SNR^2 \left( \dot{p} t + 100\right) = \left(\dot{p}t\right)^2$$ 


if we substitute $S = \dot{p} t$, it looks a little simpler
$$ S^2 - S \cdot SNR^2  - 100 SNR^2 = 0$$
then, we can use the quadratic formula to solve for the needed signal:
$$S = SNR^2 \pm \frac{\sqrt{SNR^4 + 400 SNR^2}}{2}$$

In [ ]:
SNR = 100
target_signal = SNR**2 + (SNR**4 + 4*readnoise_sum.value*SNR**2)**0.5 / 2
target_signal

In [ ]:
integration_time_including_readnoise = target_signal / phot_per_s
integration_time_including_readnoise

In [ ]:
# how much longer is that?
integration_time_including_readnoise / integration_time

So, accounting for readnoise, to get to the same SNR, we need to integrate for about twice as long (2.12x).

## What would we need to be even more accurate in our prediction?

Note that all of these calculations make a few assumptions that are generally not correct:

1. We assumed the V-filter has a uniform transmission of 1 across its band.  In reality, the V-filter transmits closer to ~75% of the total band, so the total photon rate (in any given filter) is less than what we calculated above.

2. We assumed that the CCD has perfect _quantum efficiency_.  The best CCDs have close to unity (one) efficiency, but not exactly.  Ours are probably in the 60-80% range.

3. We assumed no flux losses to the atmosphere (absorption or scattering) or the telescope optics.  We know from class that the atmosphere absorbs anywhere from 10-30% of the incident flux if we point straight up on a clear night.

Together these effects amount to a substantial loss of light.  Let's estimate how much:

In [ ]:
filter_efficiency = 0.75
quantum_efficiency = 0.7
atmosphere_loss = 0.2
received_fraction = filter_efficiency * quantum_efficiency * (1-atmosphere_loss)
received_fraction

We're getting less than 50% of the light recorded on our CCD.  That means we're liable to underestimate the brightness of our targets by a lot!  Next time, we'll go over how we account for that loss by _calibrating_.

## Do it yourself for a more reasonable target.

We noted above that P Cygni is really hard to observe, since it will saturate our detector.

Now, let's repeat the above exercises for a different target: `TYC 2788-108-1`

Answer the following questions (using the code you learned above):

1.  How bright is TYC 2788-108-1 in the [Johnson V-band](http://voservices.net/filter/filterdtls.aspx?filterid=60)?  Give your answer in magnitudes and in Janskys, assuming the zero-magnitude flux is 3500 Jy

2. Using the same telescope as above, what is our expected received flux in ergs/second and in photons per second?  Assume the V-filter band width is 870 Angstroms and the band center is 5504 Angstroms.

3. How many photons per second per pixel will the star produce if the point spread function, the area over which the star's light is spread, is the same as above ($\sigma_{Gaussian} = 2"$)?  How long can we integrate without saturating (assuming we saturate at 30,000 counts)?

4. Again following the same assumptions (the same spread among pixels, same readnoise), how long do we need to integrate to get to a signal-to-noise ratio of 100?  Account for both photon statistics and readnoise